In [1]:
import Network as mynetwork
import RunManager as rm
import RunBuilder as bm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections import OrderedDict

In [2]:
data = mynetwork.getdata('winequality-red.csv')
loader = DataLoader(data, batch_size=50, shuffle=True)
number_epochs = 1000
NN_width = 100
NN_drop = 0.1

In [3]:
params = OrderedDict(lr=[0.001], batch_size=[50], dropout=[0.1], width=[200,500])
b = bm.RunBuilder.get_runs(params)

In [4]:
for run in b:
    loader = DataLoader(data, batch_size=run.batch_size, shuffle=True)
    network = mynetwork.mynetwork(run.width,run.dropout)
    optimizer = torch.optim.Adam(network.parameters(), lr=run.lr)
    criterion = nn.MSELoss() #nn.BCELoss()
    m = rm.RunManager()
    m.begin_run(run, network, loader)
    for epoch in range(number_epochs):
        m.begin_epoch()
        for batch in loader:
            characteristics, labels = batch
            preds = network(characteristics)  # Pass Batch
            loss = criterion(preds,labels)
            optimizer.zero_grad()  # Zero Gradients
            loss.backward()  # Calculate Gradients
            optimizer.step()  # Update Weights
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()

In [ ]:
torch.save(network.state_dict(), 'mynetwork')

In [1]:
network.load_state_dict(torch.load('mynetwork', weights_only=True))


NameError: name 'network' is not defined

In [6]:
with torch.no_grad():
    network.eval()
    data = mynetwork.getevaldata('winequality-red.csv')
    loader = DataLoader(data, batch_size=50, shuffle=True)
    first_batch = next(iter(loader))
    characteristics, labels = first_batch

In [15]:
preds = network(characteristics)

In [38]:
print("|  Preds  |  Label  | Differ  |")
for I in range(10):
    if abs(preds[I].item() - labels[I].item()) >0.5:
        a= '*'
    else:
        a = '|'
    print('|%8.3f' %preds[I].item(), '|%8.3f' %labels[I].item(), '|%8.3f' %(preds[I].item() - labels[I].item()),a)

|  Preds  |  Label  | Differ  |
|   6.307 |   6.000 |   0.307 |
|   5.385 |   6.000 |  -0.615 *
|   5.738 |   6.000 |  -0.262 |
|   6.252 |   8.000 |  -1.748 *
|   5.902 |   6.000 |  -0.098 |
|   6.027 |   6.000 |   0.027 |
|   4.930 |   5.000 |  -0.070 |
|   5.795 |   6.000 |  -0.205 |
|   7.148 |   6.000 |   1.148 *
|   6.309 |   6.000 |   0.309 |
